## CUSTOMER LIFETIME VALUE (Müşteri Yaşam Boyu Değeri)


### 1. Veri Hazırlama
### 2. Average Order Value (average_order_value = total_price / total_transaction)
### 3. Purchase Frequency (total_transaction / total_number_of_customers)
### 4. Repeat Rate & Churn Rate (birden fazla alışveriş yapan müşteri sayısı / tüm müşteriler)
### 5. Profit Margin (profit_margin =  total_price * 0.10)
### 6. Customer Value (customer_value = average_order_value * purchase_frequency)
### 7. Customer Lifetime Value (CLTV = (customer_value / churn_rate) x profit_margin)
### 8. Segmentlerin Oluşturulması
### 9. BONUS: Tüm İşlemlerin Fonksiyonlaştırılması

In [2]:
##################################################
# 1. Veri Hazırlama
##################################################

In [3]:
# Veri Seti Hikayesi
# https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

# Online Retail II isimli veri seti İngiltere merkezli online bir satış mağazasının
# 01/12/2009 - 09/12/2011 tarihleri arasındaki satışlarını içeriyor.

# Değişkenler
# InvoiceNo: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. C ile başlıyorsa iptal edilen işlem.
# StockCode: Ürün kodu. Her bir ürün için eşsiz numara.
# Description: Ürün ismi
# Quantity: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.
# InvoiceDate: Fatura tarihi ve zamanı.
# UnitPrice: Ürün fiyatı (Sterlin cinsinden)
# CustomerID: Eşsiz müşteri numarası
# Country: Ülke ismi. Müşterinin yaşadığı ülke.

In [4]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [5]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [7]:
df_ = pd.read_excel("C:/Users/Erdem Yavuz/Desktop/PYTHON/CRM_Analytics/online_retail_II.xlsx", sheet_name="Year 2009-2010")

In [8]:
df = df_.copy()

In [13]:
df.head()
df.shape

(515255, 8)

In [25]:
df = df[~df["Invoice"].str.contains("C", na=False)]

In [27]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,515255.00000,10.95663,104.35401,-9600.00000,1.00000,3.00000,10.00000,19152.00000
Price,515255.00000,3.95637,127.68856,-53594.36000,1.25000,2.10000,4.21000,25111.09000
Customer ID,407695.00000,15368.50411,1679.79570,12346.00000,13997.00000,15321.00000,16812.00000,18287.00000


In [31]:
df = df[df["Quantity"] > 0]

In [32]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,513134.00000,11.71543,92.97472,1.00000,1.00000,3.00000,10.00000,19152.00000
Price,513134.00000,3.97273,127.95193,-53594.36000,1.25000,2.10000,4.21000,25111.09000
Customer ID,407695.00000,15368.50411,1679.79570,12346.00000,13997.00000,15321.00000,16812.00000,18287.00000


In [33]:
df.dropna(inplace = True)

C:\Users\Erdem Yavuz\AppData\Local\Temp\ipykernel_11196\1791587065.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace = True)


In [34]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,407695.00000,13.58669,96.84223,1.00000,2.00000,5.00000,12.00000,19152.00000
Price,407695.00000,3.29419,34.75666,0.00000,1.25000,1.95000,3.75000,10953.50000
Customer ID,407695.00000,15368.50411,1679.79570,12346.00000,13997.00000,15321.00000,16812.00000,18287.00000


In [37]:
df["TotalPrice"] = df["Quantity"] * df["Price"]
df.head()

C:\Users\Erdem Yavuz\AppData\Local\Temp\ipykernel_11196\4219876379.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["TotalPrice"] = df["Quantity"] * df["Price"]


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom,83.40000
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom,100.80000
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom,30.00000


In [44]:
cltv_c = df.groupby("Customer ID").agg({"Invoice" : lambda x : x.nunique(),
                               "Quantity" : lambda x : x.sum(),
                               "TotalPrice" : lambda x : x.sum()})

In [49]:
cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']

In [50]:
cltv_c.head()

,total_transaction,total_unit,total_price
Customer ID,,,
12346.00000,11,70,372.86000
12347.00000,2,828,1323.32000
12348.00000,1,373,222.16000
12349.00000,3,993,2671.14000
12351.00000,1,261,300.93000


In [51]:
##################################################
# 2. Average Order Value (average_order_value = total_price / total_transaction)
##################################################

In [53]:
cltv_c["avg_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]

In [54]:
cltv_c.head()

,total_transaction,total_unit,total_price,avg_order_value
Customer ID,,,,
12346.00000,11,70,372.86000,33.89636
12347.00000,2,828,1323.32000,661.66000
12348.00000,1,373,222.16000,222.16000
12349.00000,3,993,2671.14000,890.38000
12351.00000,1,261,300.93000,300.93000


In [55]:
##################################################
# 3. Purchase Frequency (total_transaction / total_number_of_customers)
##################################################

In [61]:
cltv_c["purchase_freq"] = cltv_c["total_transaction"] / cltv_c.shape[0]

In [62]:
cltv_c.head()

,total_transaction,total_unit,total_price,avg_order_value,purchase_freq
Customer ID,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255
12347.00000,2,828,1323.32000,661.66000,0.00046
12348.00000,1,373,222.16000,222.16000,0.00023
12349.00000,3,993,2671.14000,890.38000,0.00070
12351.00000,1,261,300.93000,300.93000,0.00023


In [63]:
##################################################
# 4. Repeat Rate & Churn Rate (birden fazla alışveriş yapan müşteri sayısı / tüm müşteriler)
##################################################

In [75]:
# birden fazla alışveriş yapan müşteri sayısı
cltv_c[cltv_c["total_transaction"] > 1].shape[0]
# (birden fazla alışveriş yapan müşteri sayısı / tüm müşteriler)
repeat_rate = cltv_c[cltv_c["total_transaction"] > 1].shape[0] / cltv_c.shape[0]
churn_rate = 1 - repeat_rate
churn_rate 

0.3293926750115902

In [76]:
##################################################
# 5. Profit Margin (profit_margin =  total_price * 0.10)
##################################################

In [77]:
cltv_c["profit_margin"] = cltv_c["total_price"] * 0.1

In [78]:
cltv_c.head()

,total_transaction,total_unit,total_price,avg_order_value,purchase_freq,profit_margin
Customer ID,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300


In [79]:
##################################################
# 6. Customer Value (customer_value = average_order_value * purchase_frequency)
##################################################

In [80]:
cltv_c["customer_value"] = cltv_c["avg_order_value"] * cltv_c["purchase_freq"]

In [81]:
cltv_c.head()

,total_transaction,total_unit,total_price,avg_order_value,purchase_freq,profit_margin,customer_value
Customer ID,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600,0.08643
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600,0.05150
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300,0.06976


In [82]:
##################################################
# 7. Customer Lifetime Value (CLTV = (customer_value / churn_rate) x profit_margin)
##################################################

In [83]:
cltv_c["CLTV"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["profit_margin"]

In [84]:
cltv_c.head()

,total_transaction,total_unit,total_price,avg_order_value,purchase_freq,profit_margin,customer_value,CLTV
Customer ID,,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600,0.08643,9.78357
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675,123.23546
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600,0.05150,3.47326
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918,502.11041
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300,0.06976,6.37290


In [87]:
cltv_c.sort_values(by = "CLTV" , ascending=False)

,total_transaction,total_unit,total_price,avg_order_value,purchase_freq,profit_margin,customer_value,CLTV
Customer ID,,,,,,,,
18102.00000,89,124216,349164.35000,3923.19494,0.02063,34916.43500,80.93749,8579573.77276
14646.00000,78,170342,248396.50000,3184.57051,0.01808,24839.65000,57.57916,4342070.45829
14156.00000,102,108107,196566.74000,1927.12490,0.02364,19656.67400,45.56484,2719105.08615
14911.00000,205,69722,152147.57000,742.18327,0.04752,15214.75700,35.26833,1629055.80978
13694.00000,94,125893,131443.19000,1398.33181,0.02179,13144.31900,30.46898,1215855.89003
...,...,...,...,...,...,...,...,...
15913.00000,1,3,6.30000,6.30000,0.00023,0.63000,0.00146,0.00279
13788.00000,1,1,3.75000,3.75000,0.00023,0.37500,0.00087,0.00099
14095.00000,1,1,2.95000,2.95000,0.00023,0.29500,0.00068,0.00061


In [88]:
cltv_c.describe().T

,count,mean,std,min,25%,50%,75%,max
total_transaction,4314.00000,4.45410,8.16866,1.00000,1.00000,2.00000,5.00000,205.00000
total_unit,4314.00000,1284.01113,6458.45205,1.00000,158.00000,382.00000,995.25000,220600.00000
total_price,4314.00000,2047.28866,8912.52324,0.00000,307.95000,705.55000,1722.80250,349164.35000
avg_order_value,4314.00000,378.14723,492.51721,0.00000,181.95632,286.94600,423.53375,11880.84000
purchase_freq,4314.00000,0.00103,0.00189,0.00023,0.00023,0.00046,0.00116,0.04752
profit_margin,4314.00000,204.72887,891.25232,0.00000,30.79500,70.55500,172.28025,34916.43500
customer_value,4314.00000,0.47457,2.06595,0.00000,0.07138,0.16355,0.39935,80.93749
CLTV,4314.00000,5883.60651,156068.63594,0.00000,6.67369,35.03173,208.87042,8579573.77276


In [89]:
##################################################
# 8. Segmentlerin Oluşturulması
##################################################

In [90]:
cltv_c.sort_values(by = "CLTV", ascending = False)

,total_transaction,total_unit,total_price,avg_order_value,purchase_freq,profit_margin,customer_value,CLTV
Customer ID,,,,,,,,
18102.00000,89,124216,349164.35000,3923.19494,0.02063,34916.43500,80.93749,8579573.77276
14646.00000,78,170342,248396.50000,3184.57051,0.01808,24839.65000,57.57916,4342070.45829
14156.00000,102,108107,196566.74000,1927.12490,0.02364,19656.67400,45.56484,2719105.08615
14911.00000,205,69722,152147.57000,742.18327,0.04752,15214.75700,35.26833,1629055.80978
13694.00000,94,125893,131443.19000,1398.33181,0.02179,13144.31900,30.46898,1215855.89003
...,...,...,...,...,...,...,...,...
15913.00000,1,3,6.30000,6.30000,0.00023,0.63000,0.00146,0.00279
13788.00000,1,1,3.75000,3.75000,0.00023,0.37500,0.00087,0.00099
14095.00000,1,1,2.95000,2.95000,0.00023,0.29500,0.00068,0.00061


In [91]:
cltv_c["segment"] = pd.qcut(cltv_c["CLTV"], 4, labels = ["D","C","B","A"])

In [94]:
cltv_c.head()

,total_transaction,total_unit,total_price,avg_order_value,purchase_freq,profit_margin,customer_value,CLTV,segment
Customer ID,,,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600,0.08643,9.78357,C
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675,123.23546,B
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600,0.05150,3.47326,D
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918,502.11041,A
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300,0.06976,6.37290,D


In [96]:
cltv_c.groupby("segment").agg({"count","mean","sum"})

total_transaction                 total_unit                      \
                    count    sum     mean      count      sum       mean   
segment                                                                    
D                    1079   1326  1.22892       1079   117616  109.00463   
C                    1078   2160  2.00371       1078   305135  283.05659   
B                    1078   4063  3.76902       1078   733211  680.15863   
A                    1079  11666 10.81186       1079  4383262 4062.33735   

        total_price                          avg_order_value               \
              count           sum       mean           count          sum   
segment                                                                     
D              1079  192265.13000  178.18826            1079 169805.46550   
C              1078  513016.45300  475.89652            1078 317422.35514   
B              1078 1219605.20000 1131.35918            1078 420127.12365   
A              1079 6907116.49100 6401.40546            1079 723972.22347   

                  purchase_freq                 profit_margin               \
             mean         count     sum    mean         count          sum   
segment                                                                      
D       157.37300          1079 0.30737 0.00028          1079  19226.51300   
C       294.45487          1078 0.50070 0.00046          1078  51301.64530   
B       389.72832          1078 0.94182 0.00087          1078 121960.52000   
A       670.96592          1079 2.70422 0.00251          1079 690711.64910   

                  customer_value                     CLTV                 \
             mean          count        sum    mean count            sum   
segment                                                                    
D        17.81883           1079   44.56772 0.04130  1079     2849.32790   
C        47.58965           1078  118.91897 0.11031  1078    18184.26342   
B       113.13592           1078  282.70867 0.26225  1078   103549.86329   
A       640.14055           1079 1601.09330 1.48387  1079 25257295.04118   

                     
               mean  
segment              
D           2.64071  
C          16.86852  
B          96.05739  
A       23408.05843

In [97]:
cltv_c.to_csv("cltc_c.csv")

In [98]:
##################################################
# 9. BONUS: Tüm İşlemlerin Fonksiyonlaştırılması
##################################################

In [99]:
def create_cltv_c(dataframe, profit=0.10):

    # Veriyi hazırlama
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[(dataframe['Quantity'] > 0)]
    dataframe.dropna(inplace=True)
    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
    cltv_c = dataframe.groupby('Customer ID').agg({'Invoice': lambda x: x.nunique(),
                                                   'Quantity': lambda x: x.sum(),
                                                   'TotalPrice': lambda x: x.sum()})
    cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']
    # avg_order_value
    cltv_c['avg_order_value'] = cltv_c['total_price'] / cltv_c['total_transaction']
    # purchase_frequency
    cltv_c["purchase_frequency"] = cltv_c['total_transaction'] / cltv_c.shape[0]
    # repeat rate & churn rate
    repeat_rate = cltv_c[cltv_c.total_transaction > 1].shape[0] / cltv_c.shape[0]
    churn_rate = 1 - repeat_rate
    # profit_margin
    cltv_c['profit_margin'] = cltv_c['total_price'] * profit
    # Customer Value
    cltv_c['customer_value'] = (cltv_c['avg_order_value'] * cltv_c["purchase_frequency"])
    # Customer Lifetime Value
    cltv_c['cltv'] = (cltv_c['customer_value'] / churn_rate) * cltv_c['profit_margin']
    # Segment
    cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])

    return cltv_c

In [100]:
df = df_.copy() ##eski dataframe'e dönüştürme

In [102]:
clv = create_cltv_c(df)

In [103]:
clv.head()

,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600,0.08643,9.78357,C
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200,0.30675,123.23546,B
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600,0.05150,3.47326,D
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400,0.61918,502.11041,A
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300,0.06976,6.37290,D
